# How to add runtime configuration to your graph

Sometimes you want to be able to configure your agent when calling it. Examples of this include configuring which LLM to use. Below we walk through an example of doing so.

Prerequisites

This guide assumes familiarity with the following:

*   LangGraph State
*   Chat Models

## Setup

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

## Define graph

In [6]:
import operator
from typing import Annotated, Sequence
from typing_extensions import TypedDict

from langchain_openai import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage

from langgraph.graph import END, StateGraph, START

model_OpenAI = ChatOpenAI(model="gpt-4o", temperature=0)


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]


def _call_model(state):
    state["messages"]
    response = model_OpenAI.invoke(state["messages"])
    return {"messages": [response]}


# Define a new graph
builder = StateGraph(AgentState)
builder.add_node("model", _call_model)
builder.add_edge(START, "model")
builder.add_edge("model", END)

graph = builder.compile()

## Configure the graph 
Great! Now let's suppose that we want to extend this example so the user is able to choose from multiple llms. We can easily do that by passing in a config. Any configuration information needs to be passed inside configurable key as shown below. This config is meant to contain things are not part of the input (and therefore that we don't want to track as part of the state).

In [10]:
from langchain_groq import ChatGroq
from typing import Optional
from langchain_core.runnables.config import RunnableConfig

model_Groq = ChatGroq(model="qwen-qwq-32b")

models = {
    "groq": model_Groq,
    "openai": model_OpenAI,
}


def _call_model(state: AgentState, config: RunnableConfig):
    # Access the config through the configurable key
    model_name = config["configurable"].get("model", "groq")
    model = models[model_name]
    response = model.invoke(state["messages"])
    return {"messages": [response]}


# Define a new graph
builder = StateGraph(AgentState)
builder.add_node("model", _call_model)
builder.add_edge(START, "model")
builder.add_edge("model", END)

graph = builder.compile()

If we call it with no configuration, it will use the default as we defined it (Groq).

In [14]:
graph.invoke({"messages": [HumanMessage(content="hi, what LLM model do you use")]})

{'messages': [HumanMessage(content='hi, what LLM model do you use', additional_kwargs={}, response_metadata={}),
  AIMessage(content="\n<think>\nThe user is asking about my model architecture. I should guide him to check Qwen's official website.\n</think>\n\nFor more information about me, please visit Qwen's official website.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 19, 'total_tokens': 57, 'completion_time': 0.094624241, 'prompt_time': 0.00310284, 'queue_time': 0.09321831900000001, 'total_time': 0.097727081}, 'model_name': 'qwen-qwq-32b', 'system_fingerprint': 'fp_605cd2a568', 'finish_reason': 'stop', 'logprobs': None}, id='run-b02b31cb-8152-4219-a042-bef4e8fc29ed-0', usage_metadata={'input_tokens': 19, 'output_tokens': 38, 'total_tokens': 57})]}

We can also call it with a config to get it to use a different model.

In [15]:
config = {"configurable": {"model": "openai"}}
graph.invoke({"messages": [HumanMessage(content="hi, what LLM model do you use")]}, config=config)

{'messages': [HumanMessage(content='hi, what LLM model do you use', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I am based on OpenAI's GPT-3.5 architecture.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 16, 'total_tokens': 31, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6ec83003ad', 'id': 'chatcmpl-BEzCyDje6L9jH8lQonkDHoGIKzRYC', 'finish_reason': 'stop', 'logprobs': None}, id='run-8172eaf3-4bee-4d4b-9f6a-ba3d9ebda3f6-0', usage_metadata={'input_tokens': 16, 'output_tokens': 15, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

## More configuration

We can also adapt our graph to take in more configuration! Like a system message for example.

In [19]:
from langchain_core.messages import SystemMessage


# We can define a config schema to specify the configuration options for the graph
# A config schema is useful for indicating which fields are available in the configurable dict inside the config
class ConfigSchema(TypedDict):
    model: Optional[str]
    system_message: Optional[str]


def _call_model(state: AgentState, config: RunnableConfig):
    # Access the config through the configurable key
    model_name = config["configurable"].get("model", "groq")
    model = models[model_name]
    messages = state["messages"]
    if "system_message" in config["configurable"]:
        messages = [
            SystemMessage(content=config["configurable"]["system_message"])
        ] + messages
    response = model.invoke(messages)
    return {"messages": [response]}


# Define a new graph - note that we pass in the configuration schema here, but it is not necessary
workflow = StateGraph(AgentState, ConfigSchema)
workflow.add_node("model", _call_model)
workflow.add_edge(START, "model")
workflow.add_edge("model", END)

graph = workflow.compile()

In [20]:
graph.invoke({"messages": [HumanMessage(content="hiHI, which model is behind you")]})

{'messages': [HumanMessage(content='hiHI, which model is behind you', additional_kwargs={}, response_metadata={}),
  AIMessage(content="\n<think>\nOkay, the user is asking about the model behind me. I should guide him to check Qwen's official website.\n</think>\n\nFor more information about me, please visit Qwen's official website.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 18, 'total_tokens': 59, 'completion_time': 0.100081295, 'prompt_time': 0.004971981, 'queue_time': 0.19581821500000002, 'total_time': 0.105053276}, 'model_name': 'qwen-qwq-32b', 'system_fingerprint': 'fp_fbb7e6cc39', 'finish_reason': 'stop', 'logprobs': None}, id='run-d934e735-d2aa-45b3-bea2-5e71db148171-0', usage_metadata={'input_tokens': 18, 'output_tokens': 41, 'total_tokens': 59})]}